# Task 1: Acquire, pre-process and analyze the data
## Acquiring both datasets:
Dataset 1: [Search Trends](https://github.com/google-research/open-covid-19-data/blob/master/data/exports/search_trends_symptoms_dataset/README.mdhttps://)

Dataset 2: [COVID hospitalization cases](https://github.com/google-research/open-covid-19-data)

In [ ]:
# Imports

import numpy as np
import pandas as pd
import warnings
import math
import statistics
from numpy import nanmedian, NaN
warnings.filterwarnings('ignore')


In [ ]:
# the week of 08/24/2020 for the data collection
# Load into pandas dataframes
st_df = pd.read_csv('2020_US_weekly_symptoms_dataset.csv', low_memory=False)
hp_df = pd.read_csv('aggregated_cc_by.csv', low_memory=False)

## Preprocess the datasets

## Target Weeks range: 2020-03-09 to 2020-09-21


In [ ]:
# Search trends dataset Part I

#TODO: Preprocessing, remove all symptoms that have all zero entries (clean COLUMN)
st_df = st_df.dropna(how='all', axis=1)

#Remove all rows not in the date of the week chosen (clean ROW)
st_df = st_df[st_df['date'] >= '2020-03-04']

nameList = list(st_df['sub_region_1']) #extract the region names from st_df database
nameList = list(dict.fromkeys(nameList))  #remove duplicates

In [ ]:
# Hospitalization dataset Part I

#TODO: Preprocessing

#keep the hospitalization features and delete the rest  (clean COLUMN)
hp_df = hp_df[['open_covid_region_code','region_name','date', 'hospitalized_new']]

#select the regions that match the Search trends dataset (clean ROW)
hp_df= hp_df[hp_df.region_name.isin(nameList)]

#select the regions that have the valid date range (clean ROW)
hp_df = hp_df[(hp_df['date'] >= '2020-03-09') & (hp_df['date'] <= '2020-09-27')]

hp_df.reset_index(inplace = True) 
#print(hp_df.to_string())

In [ ]:
# Hospitalization dataset Part II
# Here we want to group dates in the same week together as one date (the weekdate)
hp_df1 = hp_df
weekdate = '2020-03-09'

#This loop will update all the dates row by row
for i, n in hp_df1.iterrows():
    if (i%7 == 0):
        weekdate = n['date']  #first date of the week
    else:
        hp_df1.at[i,'date'] = weekdate

In [ ]:
#sum up the hospitalized_vew for weekly
# we are only using this hp_df2 to rid regions that have insignificant hospitalized data, such as 0 for total hospitalization
def cleanRegions(df):
    hp_df2 = df
    f = dict.fromkeys(hp_df2.columns.difference(['region_name']), 'first')
    f['hospitalized_new'] = sum
    hp_df2 = hp_df2.groupby('region_name', as_index=False).agg(f)
    hp_df2 = hp_df2[hp_df2.hospitalized_new != 0]
    print(hp_df2.to_string())
    tmplist = list(hp_df2['region_name']) 
    tmplist = list(dict.fromkeys(tmplist))  
    return(tmplist)

#this nameList will be a new regions list that removes region with total of 0 hospitalization value for all its dates
nameList2 = cleanRegions(hp_df1)

#filter hp_df1 based on the nameList2 (clean ROW)
hp_df2= hp_df1[hp_df1.region_name.isin(nameList2)]

In [ ]:
# Hospitalization dataset Part III

# merge 7 week rows into 1 and sum up the hospitalized_new data
hp_df3 = hp_df2.groupby(['region_name','date'])['hospitalized_new'].apply(sum).reset_index()

In [ ]:
# Search trends dataset Part II

# Drop unnecessary columns (open_covid region_code, country_region_code, country_region) (clean COLUMN)
st_df1 = st_df.drop(st_df.columns[[0, 1, 2]], axis=1)

# Filter st_df based on nameList2 (clean ROW)
st_df1= st_df1[st_df.sub_region_1.isin(nameList2)]
# print(st_df1)
# print(st_df1.shape)

#Filter columns so that every column have at least sp_num% of non-zero entries  (clean COLUMN)
sp_num = 0.24  #optimized ratio without tremendous loss of dataset
st_df2 = st_df1.dropna(thresh=sp_num*len(st_df), axis=1)


st_df2.reset_index(inplace = True) 
st_df2 = st_df2.drop(st_df2.columns[[0]], axis=1)

# print("after........." )
#print(st_df2.head(50))
#print(st_df2.shape)

In [ ]:
#Normalization of the Search Trend Datasets:

#Find each symptom's median
#61 symptoms
medianList = []
for (columnName, columnData) in st_df2.iteritems(): 
    if (columnName != 'sub_region_1' and columnName != 'sub_region_1_code' and columnName != 'date'):
        m =  nanmedian(columnData.values) 
#         print("my median is", m)
        s = columnData.size
        i = 0
        for i in range(s):
            v = columnData.values[i]
            if (math.isnan(v)):
                columnData.values[i] = columnData.values[i]
            else:
#                 print("my val before:", columnData.values[i])
                columnData.values[i] = columnData.values[i]/m
#                 print("my val after:", columnData.values[i])
            i = i+1

# print(st_df2.shape)
# print(st_df2)

## Merging the datasets 

In [ ]:
hpData = hp_df3["hospitalized_new"]
hpData = pd.Series(hpData)

st_df2['hospitalized_new'] = hpData.values # Merging the data_set
#print(st_df2)

In [ ]:
# Convert merged dataset into a numpy array
myarray = pd.DataFrame(st_df2).to_numpy()

# Task 2: Visualize and Cluster the Data

## Visualizing the evolution of the search frequency of popular symptoms

In [ ]:
# Imports
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
# Replace NaN values by 0s
myarray[pd.isnull(myarray)] = 0

In [ ]:
regions = myarray[:,0]
time = myarray[:,2]
features = myarray[:,3:-1].astype(float)
label = myarray[:,-1].astype(int)

nData, nFeat = features.shape

In [ ]:
# Get list of symptom names
nameSymptoms = np.array([])
for name in st_df2.columns.values[3:-1]:
    nameSymptoms = np.append(nameSymptoms, name.lstrip('symptom:'))

# print(nameSymptoms)

In [ ]:
# Find the most popular searches (rank by # of instances)
mostPop = 5

arrInst = np.count_nonzero(features, axis=0)
# Get indices of most popular searches
topSearchInd = np.sort(np.argpartition(arrInst, -mostPop)[-mostPop:])
# print(topSearchInd)

In [ ]:
# Get list of symptom names of most popular searches
namePopSymptoms = nameSymptoms[topSearchInd]
# print(namePopSymptoms)

# Get array of features of only the most popular searches
popFeatures = features[:,topSearchInd]
# print(popFeatures.shape)

In [ ]:
# Reshape search data 2D array into 3D [region, time, feature]

uniqueRegions, ctRegions = np.unique(regions, return_counts=True)
# print(dict(zip(uniqueRegions, ctRegions)))

uniqueTime, ctTime = np.unique(time, return_counts=True)
# print(dict(zip(uniqueTime, ctTime)))

nRegions = ctTime[0]
nTime = ctRegions[0]

sdArr = popFeatures.reshape(-1, nTime, mostPop)
# print(sdArr.shape)
# print(sdArr[0,:,2])

In [ ]:
# Heatmap + Slider Visualization

# Processing
dataHeatmap = np.swapaxes(sdArr, 0, 2)
hCMap = plt.cm.get_cmap('PiYG_r', 9)

# %matplotlib inline

def timeSlider(TimeSlider=0):
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.set_xticks(np.arange(nRegions))
    ax.set_yticks(np.arange(mostPop))

    ax.set_xticklabels(uniqueRegions, fontsize=15)
    ax.set_yticklabels(namePopSymptoms, fontsize=15)
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
    
    ax.set_title('Evolution of symptom search trends -- Week of ' + str(uniqueTime[TimeSlider]) + ' -- Ratio to symptom median', fontsize=20)
    im = ax.imshow(dataHeatmap[:,TimeSlider,:], vmin=0.0, vmax=2, cmap=hCMap)
    plt.colorbar(im).set_ticks([0, 1, 2])
    
    
    fig.canvas.draw()
        
widgets.interact(timeSlider, TimeSlider=(0, nTime-1, 1))

In [ ]:
for i in range(mostPop):
    fig = plt.figure(facecolor='white', figsize=(20,10))
    plt.axhline(1, color='black', lw=2)
    plt.xticks(np.arange(nTime), uniqueTime, rotation=45, ha='right', rotation_mode='anchor')
    plt.yticks(np.arange(0, 5, step=0.25))
    plt.title('Evolution of ' + namePopSymptoms[i] + ' -- Average ratio to symptom median across all regions', fontsize=20)
    plt.xlabel('Time', fontsize=15)
    plt.ylabel('Normalized search frequency', fontsize=15)
    
    avgSDArr = np.average(sdArr[:,:,i], axis=0)
    barColors = []
    for avgNum in avgSDArr:
        if avgNum > 1:
            barColors.append(hCMap(7))
        else:
            barColors.append(hCMap(1))
    
    plt.bar(x=np.arange(nTime), height=avgSDArr,width=0.9, bottom=0, color=barColors)
    plt.grid(b=True)

## Using PCA to reduce data dimensionality

In [ ]:
# Imports
from sklearn.decomposition import PCA

In [ ]:
# Perform PCA on the features
pca = PCA()
pca.fit(features)
pcaEVR = pca.explained_variance_ratio_
totalPC = len(pcaEVR)
cumulativeVar = 100*np.cumsum(pcaEVR)

In [ ]:
threshCVar = 95

nPC = np.where(cumulativeVar > threshCVar)[0][0]
cVar = cumulativeVar[nPC]
nPC += 1

# print(cVar, nPC)

In [ ]:
# Plot the cumulative variance vs # of principal components to choose #PCs
fig = plt.figure()
plt.plot(np.linspace(1, totalPC, totalPC), threshCVar*np.ones((totalPC,)), 'c--')
plt.plot(np.linspace(1, totalPC, totalPC), cumulativeVar, 'x')
plt.plot(nPC, cVar, 'o')

plt.text(nPC-3, cVar-4, 'Optimal #PCs: ' + str(nPC))
plt.text(totalPC, threshCVar, 'Cumulative Variance Threshold:' + str(threshCVar) + '%', horizontalalignment='right')

plt.xlabel("Number of Principal Components")
plt.ylabel("% Variance Explained")
plt.title("Cumulative Variance Explained vs Number of Principal Components")

In [ ]:
# We have chosen the #PCs -> now reduce the search trends dataset to this dimensionality
pcaRed = PCA(n_components=nPC)
pcaRed.fit(features)
reducedFeat = pcaRed.transform(features)

fig = plt.figure()
plt.scatter(reducedFeat[:,0], reducedFeat[:,1])
plt.xlabel("PC #1")
plt.ylabel("PC #2")
plt.title("Search Trends Dataset Reduced to " + str(nPC) + "D, Visualizing in 2D")

## Using k-means clustering to evaluate groups in search trends data

In [ ]:
# Imports
from sklearn.cluster import KMeans

In [ ]:
# Use the elbow method to determine K for the reduced data
nClusterRange = range(1, 20)
inertiaRed = []
inertiaOri = []

for clst in nClusterRange:
    kmeansRed = KMeans(n_clusters=clst, random_state=0)
    kmeansRed.fit(reducedFeat)
    inertiaRed.append(kmeansRed.inertia_)
    
    kmeansOri = KMeans(n_clusters=clst, random_state=0)
    kmeansOri.fit(features)
    inertiaOri.append(kmeansOri.inertia_)

In [ ]:
# Plot the inertia
plt.figure(figsize=(21,6))

plt.subplot(1, 2, 1)
plt.plot(nClusterRange, inertiaRed, 'x', markersize=15)
plt.plot(6, inertiaRed[6-1], 'X', markersize=20)
plt.text(6.5, inertiaRed[6-1] + 300, 'Optimal: K = 6', fontsize=16)
plt.xticks(nClusterRange)
plt.xlabel('K', fontsize=16)
plt.ylabel('Inertia', fontsize=16)
plt.title('K-Means -- Inertia of reduced data versus the number of clusters (K)', fontsize=16)

plt.subplot(1, 2, 2)
plt.plot(nClusterRange, inertiaOri, 'x', markersize=15)
plt.plot(6, inertiaOri[6-1], 'X', markersize=20)
plt.text(6.5, inertiaOri[6-1] + 300, 'Optimal: K = 6', fontsize=16)
plt.xticks(nClusterRange)
plt.xlabel('K', fontsize=16)
plt.ylabel('Inertia', fontsize=16)
plt.title('K-Means -- Inertia of unreduced (original) data versus the number of clusters (K)', fontsize=16)

plt.show()

In [ ]:
# Use the optimal K (nClusters):
nClusters = 6

# Clustering on PCA-reduced data
kmeansRed = KMeans(n_clusters=nClusters, random_state=0)
kmeansRed.fit(reducedFeat)
y_PredRed = kmeansRed.predict(reducedFeat)

# Clustering on original (non-reduced) data
kmeansOri = KMeans(n_clusters=nClusters, random_state=0)
kmeansOri.fit(features)
y_PredOri = kmeansOri.predict(features)

In [ ]:
# Plot clustering results for both reduced and unreduced data

fig = plt.figure(figsize=(18,6))

plt.subplot(1,2,1)
plt.scatter(reducedFeat[:,0], reducedFeat[:,1], c=y_PredRed, cmap=plt.cm.get_cmap('viridis', nClusters))
plt.colorbar(ticks=range(nClusters))
plt.xlabel("PC #1", fontsize=16)
plt.ylabel("PC #2", fontsize=16)
plt.title("Cluster Labels for " + str(nPC) + "D K-Means with " + str(nClusters) + " Clusters", fontsize=16)

plt.subplot(1,2,2)
plt.scatter(reducedFeat[:,0], reducedFeat[:,1], c=y_PredOri, cmap=plt.cm.get_cmap('viridis', nClusters))
plt.colorbar(ticks=range(nClusters))
plt.xlabel("PC #1", fontsize=16)
plt.ylabel("PC #2", fontsize=16)
plt.title("Cluster Labels for " + str(totalPC) + "D K-Means (Unreduced Data) with " + str(nClusters) + " Clusters", fontsize=16)

# Task 3: Supervised Learning

In [ ]:
#Imports
from sklearn import neighbors
from sklearn import tree
from  sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import time

## Data preprocessing

In [ ]:
#replace NaN values to be 0's
myarray[pd.isnull(myarray)] = 0
print(myarray.shape)

#data_r is sorted based on regions
data_r = myarray
#data_t is sorted based on time
data_t = np.array(sorted(myarray,key = lambda x: x[2]))

#split the region-based data to be regions, features, and label
regions_r = data_r[:,0]
features_r = data_r[:,3:-1].astype(float)
label_r = data_r[:,-1].astype(int)
#split the time-based data to be time, features, and label
time_t = data_t[:,2]
features_t = data_t[:,3:-1].astype(float)
label_t = data_t[:,-1].astype(int)

## Region-based cross-validation (5-fold)

In [ ]:
#split the region-based data into 5 folds based on regions
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(n_splits=5)
#specify the group indices by regions
groups = regions_r
for i in range(groups.shape[0]):
    if groups[i] == 'Hawaii' or groups[i] ==  'Idaho':
        groups[i] = 1
    elif groups[i] == 'Maine' or groups[i] ==  'Montana':
        groups[i] = 2
    elif groups[i] ==  'North Dakota' or groups[i] ==  'Nebraska':
        groups[i] = 3
    elif groups[i] ==  'New Hampshire' or groups[i] ==  'New Mexico':
        groups[i] = 4
    else:
        groups[i] = 5

### KNN (region-based validation)

In [ ]:
#plot cross-validated MSE error of KNN with respect to different values of k
import matplotlib.pyplot as plt
%matplotlib notebook
k_range_r = range(1, 18)
k_scores_r = []
for k in k_range_r:
    knn_r = neighbors.KNeighborsRegressor(n_neighbors=k)
    loss = abs(cross_val_score(knn_r, features_r,label_r, cv=gkf.split(features_r,label_r, groups), scoring='neg_mean_squared_error'))
    k_scores_r.append(loss.mean())
plt.plot(k_range_r, k_scores_r)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated MSE')
plt.show()

In [ ]:
#k=11 can represent the performance of KNN
knn_r = neighbors.KNeighborsRegressor(n_neighbors=11)

start = time.time()
loss = abs(cross_val_score(knn_r, features_r,label_r, cv=gkf.split(features_r,label_r, groups), scoring='neg_mean_squared_error'))
stop = time.time()

knn_r_error = loss.mean()
train_time = stop - start
print("The error for KNN (region-based validation) is " + str(knn_r_error) + ".\n" +
      "The training time for KNN (region-based validation) is " + str(train_time) + ".")

### Decision tree (region-based validation)

In [ ]:
#plot cross-validated MSE error of decision tree with respect to different values of max_depth
import matplotlib.pyplot as plt
%matplotlib notebook
max_depth_range_r = range(1, 30)
max_depth_scores_r = []
for max_depth in max_depth_range_r:
    dt_r = tree.DecisionTreeRegressor(max_depth=max_depth, random_state=1)
    loss = abs(cross_val_score(dt_r, features_r,label_r, cv=gkf.split(features_r,label_r, groups), scoring='neg_mean_squared_error'))
    max_depth_scores_r.append(loss.mean())
plt.plot(max_depth_range_r, max_depth_scores_r)
plt.xlabel('Value of max_depth for decision tree')
plt.ylabel('Cross-Validated MSE')
plt.show()

In [ ]:
#max_depth = 2 can represent the performance of decision tree
dt_r = tree.DecisionTreeRegressor(max_depth = 2, random_state=1)

start = time.time()
loss = abs(cross_val_score(dt_r, features_r, label_r, cv = gkf.split(features_r,label_r, groups), scoring='neg_mean_squared_error'))
stop = time.time()

dt_r_error = loss.mean()
train_time = stop - start
print("The error for decision tree (region-based validation) is " + str(dt_r_error) + ".\n" +
      "The training time for decision tree (region-based validation) is " + str(train_time) + ".")

## Time-based validation

In [ ]:
#split the data into train and test sets based on the timepoint '2020-08-10'
#data before '2020-08-10' are in train set and the rest are in test set
#time_t[242] is the first '2020-08-10' which is the splitting point
features_t_train = features_t[0:241]
label_t_train = label_t[0:241]
features_t_test = features_t[241:320]
label_t_test = label_t[241:320]

### KNN (time-based validation)

In [ ]:
#plot validated MSE error of KNN with respect to different values of k
import matplotlib.pyplot as plt
%matplotlib notebook
k_range_t = range(1, 18)
k_scores_t= []
for k in k_range_t:
    knn_t = neighbors.KNeighborsRegressor(n_neighbors = k)
    knn_t.fit(features_t_train, label_t_train)
    label_t_pred_knn = knn_t.predict(features_t_test)
    loss = mean_squared_error(label_t_test, label_t_pred_knn)
    k_scores_t.append(loss)
plt.plot(k_range_t, k_scores_t)
plt.xlabel('Value of K for KNN')
plt.ylabel('Validated MSE')
plt.show()

In [ ]:
#k=15 can represent the performance of KNN
knn_t = neighbors.KNeighborsRegressor(n_neighbors=15)

start = time.time()
knn_t.fit(features_t_train, label_t_train)
label_t_pred_knn = knn_t.predict(features_t_test)
knn_t_error = mean_squared_error(label_t_test, label_t_pred_knn)
stop = time.time()

train_time = stop - start
print("The error for KNN (time-based validation) is " + str(knn_t_error) + ".\n" +
      "The training time for KNN (time-based validation) is " + str(train_time) + ".")

### Decision tree (time-based validation)

In [ ]:
#plot validated MSE error of decision tree with respect to different values of max_depth
import matplotlib.pyplot as plt
%matplotlib notebook
max_depth_range_t = range(1, 30)
max_depth_scores_t = []
for max_depth in max_depth_range_t:
    dt_t = tree.DecisionTreeRegressor(max_depth=max_depth, random_state=1)
    dt_t.fit(features_t_train, label_t_train)
    label_t_pred_dt = dt_t.predict(features_t_test)
    loss = mean_squared_error(label_t_test, label_t_pred_dt)
    max_depth_scores_t.append(loss)
plt.plot(max_depth_range_t, max_depth_scores_t)
plt.xlabel('Value of max_depth for decision tree')
plt.ylabel('Validated MSE')
plt.show()

In [ ]:
#max_depth = 1 can represent the performance of decision tree
dt_t = tree.DecisionTreeRegressor(max_depth = 1, random_state=1)

start = time.time()
dt_t.fit(features_t_train, label_t_train)
label_t_pred_dt = dt_t.predict(features_t_test)
dt_t_error = mean_squared_error(label_t_test, label_t_pred_dt)
stop = time.time()

train_time = stop - start
print("The error for decison tree (time-based validation) is " + str(dt_t_error) + ".\n" +
      "The training time for decison tree (time-based validation) is " + str(train_time) + ".")

## Optional part: learn seperate model for each region with time-based validation

### Regional KNN models (time-based validation)

In [ ]:
#plot average validated KNN MSE error of all regions with respect to different values of k
import matplotlib.pyplot as plt
%matplotlib notebook
k_range_regional = range(1, 18)
k_scores_regional= []

for k in k_range_regional:
    regional_MSE_knn = []
    for i in range(0,groups.shape[0],29):
        #each iteration deal with data from one city
        regional_features = features_r[i:i+29]
        regional_label = label_r[i:i+29]
        #split the data into train and test sets based on the timepoint 2020-08-10
        #data before 2020-08-10 are in train set and the rest are in test set
        #for data of each region, the 24th. datum is of '2020-08-10' which is the splitting point
        regional_features_train = regional_features[0:23]
        regional_label_train = regional_label[0:23]
        regional_features_test = regional_features[23:29]
        regional_label_test = regional_label[23:29]
        #calculate the KNN MSE of the city of this iteration
        knn = neighbors.KNeighborsRegressor(n_neighbors = k)
        knn.fit(regional_features_train, regional_label_train)
        regional_label_pred_knn = knn.predict(regional_features_test)
        loss_knn = mean_squared_error(regional_label_test, regional_label_pred_knn)
        regional_MSE_knn.append(loss_knn)
    #append the average validated MSE of the current k
    regional_MSE_knn = np.array(regional_MSE_knn)
    k_scores_regional.append(regional_MSE_knn.mean())

plt.plot(k_range_regional, k_scores_regional)
plt.xlabel('Value of K for KNN')
plt.ylabel('Average Validated MSE')
plt.show()

In [ ]:
#k=15 can represent the performance of KNN
regional_MSE_knn = []

start = time.time()
for i in range(0,groups.shape[0],29):
    #each iteration deal with data from one city
    regional_features = features_r[i:i+29]
    regional_label = label_r[i:i+29]
    #split the data into train and test sets based on the timepoint 2020-08-10
    #data before 2020-08-10 are in train set and the rest are in test set
    #for data of each region, the 24th. datum is of '2020-08-10' which is the splitting point
    regional_features_train = regional_features[0:23]
    regional_label_train = regional_label[0:23]
    regional_features_test = regional_features[23:29]
    regional_label_test = regional_label[23:29]
    #calculate the KNN MSE of the city of this iteration
    knn = neighbors.KNeighborsRegressor(n_neighbors = 15)
    knn.fit(regional_features_train, regional_label_train)
    regional_label_pred_knn = knn.predict(regional_features_test)
    loss_knn = mean_squared_error(regional_label_test, regional_label_pred_knn)
    regional_MSE_knn.append(loss_knn)
stop = time.time()

train_time = stop - start
regional_MSE_knn = np.array(regional_MSE_knn)
knn_regional_error = regional_MSE_knn.mean()
print("The error for regional KNN models (time-based validation) is " + str(knn_regional_error) + ".\n" +
      "The training time for regional KNN models (time-based validation) is " + str(train_time) + ".")

### Regional decision tree models (time-based validation)

In [ ]:
#plot average validated decision tree MSE error of all regions with respect to different values of max_depth
import matplotlib.pyplot as plt
%matplotlib notebook
max_depth_range_regional = range(1, 30)
dt_scores_regional= []

for max_depth in max_depth_range_regional:
    regional_MSE_dt = []
    for i in range(0,groups.shape[0],29):
        #each iteration deal with data from one city
        regional_features = features_r[i:i+29]
        regional_label = label_r[i:i+29]
        #split the data into train and test sets based on the timepoint 2020-08-10
        #data before 2020-08-10 are in train set and the rest are in test set
        #for data of each region, the 24th. datum is of '2020-08-10' which is the splitting point
        regional_features_train = regional_features[0:23]
        regional_label_train = regional_label[0:23]
        regional_features_test = regional_features[23:29]
        regional_label_test = regional_label[23:29]
        #calculate the KNN MSE of the city of this iteration
        dt = tree.DecisionTreeRegressor(max_depth = max_depth, random_state=1)
        dt.fit(regional_features_train, regional_label_train)
        regional_label_pred_dt = dt.predict(regional_features_test)
        loss_dt = mean_squared_error(regional_label_test, regional_label_pred_dt)
        regional_MSE_dt.append(loss_dt)
    #append the average validated MSE of the current k
    regional_MSE_dt = np.array(regional_MSE_dt)
    dt_scores_regional.append(regional_MSE_dt.mean())

plt.plot(max_depth_range_regional, dt_scores_regional)
plt.xlabel('Value of max_depth for decision tree')
plt.ylabel('Average Validated MSE')
plt.show()

In [ ]:
#max_depth = 2 can represent the performance of decision tree
regional_MSE_dt = []

start = time.time()
for i in range(0,groups.shape[0],29):
    #each iteration deal with data from one city
    regional_features = features_r[i:i+29]
    regional_label = label_r[i:i+29]
    #split the data into train and test sets based on the timepoint 2020-08-10
    #data before 2020-08-10 are in train set and the rest are in test set
    #for data of each region, the 24th. datum is of '2020-08-10' which is the splitting point
    regional_features_train = regional_features[0:23]
    regional_label_train = regional_label[0:23]
    regional_features_test = regional_features[23:29]
    regional_label_test = regional_label[23:29]
    #calculate the decision tree MSE of the city of this iteration
    dt = tree.DecisionTreeRegressor(max_depth = 2, random_state=1)
    dt.fit(regional_features_train, regional_label_train)
    regional_label_pred_dt = dt.predict(regional_features_test)
    loss_dt = mean_squared_error(regional_label_test, regional_label_pred_dt)
    regional_MSE_dt.append(loss_dt)
stop = time.time()

train_time = stop - start
regional_MSE_dt = np.array(regional_MSE_dt)
dt_regional_error = regional_MSE_dt.mean()
print("The error for regional decision tree models (time-based validation) is " + str(dt_regional_error) + ".\n" +
      "The training time for regional decision tree models (time-based validation) is " + str(train_time) + ".")